# 1. Download data

In [1]:
# pip install beautifulsoup4
# pip install lxml
# pip install geopy

import bs4 as bs
import urllib.request
import pandas as pd 
import numpy as np 

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

print('Libraries imported.')

Libraries imported.


### extract data

In [2]:
table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
for df in dfs:
    print(df)


[]
['M1A\n', 'Not assigned\n', 'Not assigned\n']
['M2A\n', 'Not assigned\n', 'Not assigned\n']
['M3A\n', 'North York\n', 'Parkwoods\n']
['M4A\n', 'North York\n', 'Victoria Village\n']
['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n']
['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n']
['M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n"]
['M8A\n', 'Not assigned\n', 'Not assigned\n']
['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n']
['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n']
['M2B\n', 'Not assigned\n', 'Not assigned\n']
['M3B\n', 'North York\n', 'Don Mills\n']
['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n']
['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n']
['M6B\n', 'North York\n', 'Glencairn\n']
['M7B\n', 'Not assigned\n', 'Not assigned\n']
['M8B\n', 'Not assigned\n', 'Not assigned\n']
['M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n']


### Ignore cells with a borough that is Not assigned; More than one neighborhood can exist in one postal code area

In [3]:
dft = dfs[0]
dft2 = dft[~dft.Borough.str.contains("Not assigned")]
dft2 = dft2.rename(columns = {'Postal Code':'PostalCode'}).reset_index(drop=True)
dft2.groupby('PostalCode')

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
mask = dft2['Neighborhood'] == "Not assigned"
dft2.loc[mask, 'Neighborhood'] = dft2.loc[mask, 'Borough']
dft2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
dft2.shape

(103, 3)

# Using the .csv file to join the geospatial data to the PostalCode file

In [6]:
gis = pd.read_csv("http://cocl.us/Geospatial_data")
gis = gis.rename(columns = {'Postal Code':'PostalCode'}).reset_index(drop=True)
gis.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
dfn = pd.merge(dft2,gis)

In [8]:
dfn.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
